In [1]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_core.memory import ListMemory, MemoryContent, MemoryMimeType
from autogen_ext.models.openai import OpenAIChatCompletionClient

In [2]:
# Initialize user memory
user_memory = ListMemory()

# Add user preferences to memory
await user_memory.add(MemoryContent(content="The weather should be in metric units", mime_type=MemoryMimeType.TEXT))

await user_memory.add(MemoryContent(content="Meal recipe must be vegan", mime_type=MemoryMimeType.TEXT))


async def get_weather(city: str, units: str = "imperial") -> str:
    if units == "imperial":
        return f"The weather in {city} is 73 °F and Sunny."
    elif units == "metric":
        return f"The weather in {city} is 23 °C and Sunny."
    else:
        return f"Sorry, I don't know the weather in {city}."

from autogen_core.models import ModelFamily
model_client = OpenAIChatCompletionClient(
    model="qwen2.5-coder:7b",
    # model="mistral-nemo:12b",
    base_url="http://localhost:11434/v1",
    api_key="placeholder",
    model_info={
        "vision": False,
        "function_calling": True,
        "json_output": True,
        "family": ModelFamily.ANY,
    },
)

assistant_agent = AssistantAgent(
    name="assistant_agent",
    model_client=model_client,
    tools=[get_weather],
    memory=[user_memory],
)

In [3]:
# Run the agent with a task.
stream = assistant_agent.run_stream(task="What is the weather in New York?")
await Console(stream, output_stats=True)

---------- user ----------
What is the weather in New York?
---------- assistant_agent ----------
[MemoryContent(content='The weather should be in metric units', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None), MemoryContent(content='Meal recipe must be vegan', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None)]
---------- assistant_agent ----------
[FunctionCall(id='call_n18j6t9c', arguments='{"city":"New York","units":"metric"}', name='get_weather')]
[Prompt tokens: 200, Completion tokens: 31]
---------- assistant_agent ----------
[FunctionExecutionResult(content='The weather in New York is 23 °C and Sunny.', name='get_weather', call_id='call_n18j6t9c', is_error=False)]
---------- assistant_agent ----------
The weather in New York is 23 °C and Sunny.
---------- Summary ----------
Number of messages: 5
Finish reason: None
Total prompt tokens: 200
Total completion tokens: 31
Duration: 2.57 seconds


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='What is the weather in New York?', type='TextMessage'), MemoryQueryEvent(source='assistant_agent', models_usage=None, metadata={}, content=[MemoryContent(content='The weather should be in metric units', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None), MemoryContent(content='Meal recipe must be vegan', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None)], type='MemoryQueryEvent'), ToolCallRequestEvent(source='assistant_agent', models_usage=RequestUsage(prompt_tokens=200, completion_tokens=31), metadata={}, content=[FunctionCall(id='call_n18j6t9c', arguments='{"city":"New York","units":"metric"}', name='get_weather')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='assistant_agent', models_usage=None, metadata={}, content=[FunctionExecutionResult(content='The weather in New York is 23 °C and Sunny.', name='get_weather', call_id='call_n18j6t9c', is_error=False)], 

In [4]:
await assistant_agent._model_context.get_messages()

[UserMessage(content='What is the weather in New York?', source='user', type='UserMessage'),
 SystemMessage(content='\nRelevant memory content (in chronological order):\n1. The weather should be in metric units\n2. Meal recipe must be vegan\n', type='SystemMessage'),
 AssistantMessage(content=[FunctionCall(id='call_n18j6t9c', arguments='{"city":"New York","units":"metric"}', name='get_weather')], thought=None, source='assistant_agent', type='AssistantMessage'),
 FunctionExecutionResultMessage(content=[FunctionExecutionResult(content='The weather in New York is 23 °C and Sunny.', name='get_weather', call_id='call_n18j6t9c', is_error=False)], type='FunctionExecutionResultMessage')]

In [5]:
stream = assistant_agent.run_stream(task="Write brief meal recipe with broth")
await Console(stream, output_stats=True)

---------- user ----------
Write brief meal recipe with broth
---------- assistant_agent ----------
[MemoryContent(content='The weather should be in metric units', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None), MemoryContent(content='Meal recipe must be vegan', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None)]
---------- assistant_agent ----------
Here's a simple vegan meal recipe using broth:

### Vegan Broccoli Soup

#### Ingredients:
- 2 cups frozen broccoli florets
- 1 onion, finely chopped
- 2 cloves garlic, minced
- 4 cups vegetable broth (low-sodium if possible)
- ½ teaspoon salt
- ¼ teaspoon black pepper
- A dash of fresh parsley for garnish (optional)

#### Instructions:
1. In a large pot, heat the broth over low Heat.
2. Add the onion and garlic and cook until soft and fragrant, about 5 minutes.
3. Add the frozen broccoli and cook for another 5-7 minutes or until tender.
4. Season with salt and black pepper to taste.
5. If needed, add a splash mor

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write brief meal recipe with broth', type='TextMessage'), MemoryQueryEvent(source='assistant_agent', models_usage=None, metadata={}, content=[MemoryContent(content='The weather should be in metric units', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None), MemoryContent(content='Meal recipe must be vegan', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None)], type='MemoryQueryEvent'), TextMessage(source='assistant_agent', models_usage=RequestUsage(prompt_tokens=293, completion_tokens=181), metadata={}, content="Here's a simple vegan meal recipe using broth:\n\n### Vegan Broccoli Soup\n\n#### Ingredients:\n- 2 cups frozen broccoli florets\n- 1 onion, finely chopped\n- 2 cloves garlic, minced\n- 4 cups vegetable broth (low-sodium if possible)\n- ½ teaspoon salt\n- ¼ teaspoon black pepper\n- A dash of fresh parsley for garnish (optional)\n\n#### Instructions:\n1. In a large pot, h

In [6]:
stream = assistant_agent.run_stream(task="请用中文翻译以上回复")
await Console(stream, output_stats=True)

---------- user ----------
请用中文翻译以上回复
---------- assistant_agent ----------
[MemoryContent(content='The weather should be in metric units', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None), MemoryContent(content='Meal recipe must be vegan', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None)]
---------- assistant_agent ----------
以下是使用汤的基本的素食餐食配方：

### 蔬菜汤

#### 配料：
- 2杯冻西兰花
- 一个切成细丝的大葱
- 两瓣大蒜，剁碎
- 4杯蔬菜汤（尽量选择低钠的）
- 半茶匙食盐
- 四分之一茶匙黑胡椒
- 一些新鲜的欧芹作为装饰（可选）

#### 做法：
1. 在一个大锅中，用小火加热鸡汤。
2. 加入大葱和大蒜并煮至软且香，约5分钟。
3. 添加冻西兰花并煮5到7分钟或直到变熟。
4. 根据口味添加食盐和黑胡椒调味。
5. 如果需要，可加入一些汤以达到所需的质地。
6. 用新鲜的欧芹装饰，如果使用的话。

现在即可享用你的素食西兰花汤！
[Prompt tokens: 516, Completion tokens: 202]
---------- Summary ----------
Number of messages: 3
Finish reason: None
Total prompt tokens: 516
Total completion tokens: 202
Duration: 11.86 seconds


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='请用中文翻译以上回复', type='TextMessage'), MemoryQueryEvent(source='assistant_agent', models_usage=None, metadata={}, content=[MemoryContent(content='The weather should be in metric units', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None), MemoryContent(content='Meal recipe must be vegan', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None)], type='MemoryQueryEvent'), TextMessage(source='assistant_agent', models_usage=RequestUsage(prompt_tokens=516, completion_tokens=202), metadata={}, content='以下是使用汤的基本的素食餐食配方：\n\n### 蔬菜汤\n\n#### 配料：\n- 2杯冻西兰花\n- 一个切成细丝的大葱\n- 两瓣大蒜，剁碎\n- 4杯蔬菜汤（尽量选择低钠的）\n- 半茶匙食盐\n- 四分之一茶匙黑胡椒\n- 一些新鲜的欧芹作为装饰（可选）\n\n#### 做法：\n1. 在一个大锅中，用小火加热鸡汤。\n2. 加入大葱和大蒜并煮至软且香，约5分钟。\n3. 添加冻西兰花并煮5到7分钟或直到变熟。\n4. 根据口味添加食盐和黑胡椒调味。\n5. 如果需要，可加入一些汤以达到所需的质地。\n6. 用新鲜的欧芹装饰，如果使用的话。\n\n现在即可享用你的素食西兰花汤！', type='TextMessage')], stop_reason=None)

In [7]:
import os
from pathlib import Path

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_core.memory import MemoryContent, MemoryMimeType
from autogen_ext.memory.chromadb import ChromaDBVectorMemory, PersistentChromaDBVectorMemoryConfig
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Initialize ChromaDB memory with custom config
chroma_user_memory = ChromaDBVectorMemory(
    config=PersistentChromaDBVectorMemoryConfig(
        collection_name="preferences",
        persistence_path=os.path.join(str(Path.home()), ".chromadb_autogen"),
        k=2,  # Return top  k results
        score_threshold=0.4,  # Minimum similarity score
    )
)
# a HttpChromaDBVectorMemoryConfig is also supported for connecting to a remote ChromaDB server

# Add user preferences to memory
await chroma_user_memory.add(
    MemoryContent(
        content="The weather should be in metric units",
        mime_type=MemoryMimeType.TEXT,
        metadata={"category": "preferences", "type": "units"},
    )
)

await chroma_user_memory.add(
    MemoryContent(
        content="Meal recipe must be vegan",
        mime_type=MemoryMimeType.TEXT,
        metadata={"category": "preferences", "type": "dietary"},
    )
)


# Create assistant agent with ChromaDB memory
assistant_agent = AssistantAgent(
    name="assistant_agent",
    model_client=model_client,
    tools=[get_weather],
    memory=[user_memory],
)

stream = assistant_agent.run_stream(task="What is the weather in New York?")
await Console(stream, output_stats=True)

await user_memory.close()

/Users/kevinlights/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:13<00:00, 6.06MiB/s]


---------- user ----------
What is the weather in New York?
---------- assistant_agent ----------
[MemoryContent(content='The weather should be in metric units', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None), MemoryContent(content='Meal recipe must be vegan', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None)]
---------- assistant_agent ----------
[FunctionCall(id='call_ddy93dos', arguments='{"city":"New York","units":"metric"}', name='get_weather')]
[Prompt tokens: 200, Completion tokens: 31]
---------- assistant_agent ----------
[FunctionExecutionResult(content='The weather in New York is 23 °C and Sunny.', name='get_weather', call_id='call_ddy93dos', is_error=False)]
---------- assistant_agent ----------
The weather in New York is 23 °C and Sunny.
---------- Summary ----------
Number of messages: 5
Finish reason: None
Total prompt tokens: 200
Total completion tokens: 31
Duration: 2.26 seconds


In [8]:
chroma_user_memory.dump_component().model_dump_json()

'{"provider":"autogen_ext.memory.chromadb.ChromaDBVectorMemory","component_type":"memory","version":1,"component_version":1,"description":"Store and retrieve memory using vector similarity search powered by ChromaDB.","label":"ChromaDBVectorMemory","config":{"client_type":"persistent","collection_name":"preferences","distance_metric":"cosine","k":2,"score_threshold":0.4,"allow_reset":false,"tenant":"default_tenant","database":"default_database","persistence_path":"/Users/kevinlights/.chromadb_autogen"}}'